In [1]:
import NegativeClassOptimization.config as config
import numpy as np
import pandas as pd
import glob
from pathlib import Path
import re
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
import NegativeClassOptimization as ml

In [4]:
import torch.optim as optim

In [5]:
config.ANTIGENS

['3VRL',
 '1NSN',
 '3RAJ',
 '5E94',
 '1H0D',
 '1WEJ',
 '1ADQ',
 '1FBI',
 '2YPV',
 '1OB1']

In [70]:
perf_df = pd.read_csv('../data/openset_performance.tsv', sep='\t')
perf_df[['train_task', 'train_ag', 'train_neg', 'seed_id', 'split_id']] = perf_df.task_1.str.split('__', expand=True)
perf_df[['test_task', 'test_ag', 'test_neg', 'test_seed_id', 'test_split_id']] = perf_df.task_2.str.split('__', expand=True)
perf_df.drop(['task_1', 'task_2'], inplace=True, axis=1)
perf_df.head()

,acc,recall,precision,f1,train_task,train_ag,train_neg,seed_id,split_id,test_task,test_ag,test_neg,test_seed_id,test_split_id
0,0.972986,0.975,0.971116,0.973054,ONE_VS_NINE,3VRL,auto,0,42,ONE_VS_NINE,3VRL,auto,0,42
1,0.987000,0.975,0.998975,0.986842,ONE_VS_NINE,3VRL,auto,0,42,HIGH_VS_95LOW,3VRL,auto,0,42
2,0.882900,0.975,0.823341,0.892775,ONE_VS_NINE,3VRL,auto,0,42,HIGH_VS_LOOSER,3VRL,auto,0,42
3,0.912356,0.998,0.852117,0.919307,HIGH_VS_95LOW,3VRL,auto,0,42,ONE_VS_NINE,3VRL,auto,0,42
4,0.996600,0.998,0.995213,0.996605,HIGH_VS_95LOW,3VRL,auto,0,42,HIGH_VS_95LOW,3VRL,auto,0,42


In [49]:
#choosing the best model
def get_best_model_seed_split(df, train_task, test_task, ag, metric):
    task_df = df[df['train_task'] == train_task][df['test_task'] == test_task][df['train_ag'] == ag]
    best_row = task_df[task_df['acc'] == task_df['acc'].max()]
    seed_id = best_row['seed_id'].values[0]
    split_id = best_row['split_id'].values[0]
    return seed_id , split_id

In [68]:
tasks = ['ONE_VS_NINE', 'HIGH_VS_95LOW', 'HIGH_VS_LOOSER']
tasks_dict = {'ONE_VS_NINE': '1v9', 'HIGH_VS_95LOW': 'high_vs_95low', 'HIGH_VS_LOOSER': 'high_vs_looser'}
task_template = {'ONE_VS_NINE': '{ag}__vs__9', 'HIGH_VS_95LOW': '{ag}_high__vs__{ag}_95low', 'HIGH_VS_LOOSER': '{ag}_high__vs__{ag}_looser'}

In [64]:
ag = '3VRL'
for task in tasks:
    seed_id, split_id = get_best_model_seed_split(perf_df, task, task, ag, 'acc')
    
    model_path = f"./torch_models/Frozen_MiniAbsolut_ML/{tasks_dict[task]}/seed_{seed_id}/split_{split_id}/{task_template[task].format(ag=ag)}/swa_model/data/model.pth"
    model = torch.load(model_path, map_location=torch.device('cpu'))
    break

/var/folders/ww/bptfq9n12_jbyy78qcjp7s4c0000gn/T/ipykernel_12160/1323516425.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  task_df = df[df['train_task'] == train_task][df['test_task'] == test_task][df['train_ag'] == ag]


In [67]:
model

AveragedModel(
  (module): SNN(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (linear_1): Linear(in_features=220, out_features=10, bias=True)
    (activation): ReLU()
    (linear_2): Linear(in_features=10, out_features=1, bias=True)
    (final): Sigmoid()
  )
)

In [63]:
#swa_model = optim.swa_utils.AveragedModel(model)


In [66]:
swa_model

AveragedModel(
  (module): AveragedModel(
    (module): SNN(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (linear_1): Linear(in_features=220, out_features=10, bias=True)
      (activation): ReLU()
      (linear_2): Linear(in_features=10, out_features=1, bias=True)
      (final): Sigmoid()
    )
  )
)